In [74]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

print("Generating Item Similarity Matrix...")
Description = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "Item_description_Ponds")
Description['merged'] = Description['Description'].astype(str) + " " + Description['Ingredients'].astype(str) + " " + Description['Work'].astype(str) #+ " " + Description['Tutorial'].astype(str) + Description['Title_1'].astype(str) + ", " + Description['Title_2'].astype(str) + ", " + Description['Title_3'].astype(str) + ", " + Description['Title_4'].astype(str)  + ", " + Description['Title_5'].astype(str) 

Description_name = Description['Name'].tolist()
Description_desc = Description['merged'].tolist()
Description_clean = []
for i in Description_desc:
    Description_clean.append(i.replace('\n', ' ').replace('\r', ' ').replace('\\', ' ').replace('nan', ' ').replace('/', ' '))

similar_products = pd.DataFrame({'Name': Description_name})
for ii,X in enumerate(Description_clean):
    similarity_index = []
    for j,Y in enumerate(Description_clean):
        #Removing symbols
        symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
        for i in symbols:
            Y = Y.replace(i, ' ')
            X = X.replace(i, ' ')

        X_list = word_tokenize(X.lower())  
        Y_list = word_tokenize(Y.lower()) 

        # sw contains the list of stopwords 
        sw = stopwords.words('english')  
        l1 =[];l2 =[] 

        # remove stop words from the string 
        X_set = {w for w in X_list if not w in sw}  
        Y_set = {w for w in Y_list if not w in sw} 

        #Stemming and remove words with length 1 and 2 letters
        X_set_new, Y_set_new = {''}, {''}
        for w in X_set:
            if len(w) > 2:
                X_set_new.add(stemmer.stem(w))
        for w in Y_set:
            if len(w) > 2:
                Y_set_new.add(stemmer.stem(w))    

        # form a set containing keywords of both strings  
        rvector = X_set_new.union(Y_set_new)  
        for w in rvector: 
            if w in X_set_new: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set_new: l2.append(1) 
            else: l2.append(0) 
        c = 0

        # cosine formula  
        for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        similarity_index.append(round(cosine, 2))

    similar_products[f'{Description_name[ii]}'] = similarity_index
similar_products.to_excel("Similarity_matrix_output_Ponds.xlsx", index=False)
similar_products

Generating Item Similarity Matrix...


,Name,Loción Bio-Hydratante,Loción C Dual,Agua Micelar C,Crema Bio-Hydratante,Crema Clarant B3 Normal a Grasa,Crema C con extracto de pepino,Crema de día Rejuveness,Jabón Facial Oil Control,Jabón Facial Sensitive,Crema H con filtro solar,Crema S Nutritiva,Crema Sensitive con extracto de manzanilla,Crema de noche Rejuveness,Toallitas Sensitive,Toallitas C
0,Loción Bio-Hydratante,1.00,0.40,0.29,0.41,0.22,0.26,0.23,0.34,0.27,0.23,0.19,0.20,0.24,0.26,0.29
1,Loción C Dual,0.40,1.00,0.29,0.29,0.19,0.35,0.34,0.31,0.24,0.30,0.17,0.22,0.28,0.31,0.40
2,Agua Micelar C,0.29,0.29,1.00,0.13,0.19,0.19,0.16,0.28,0.41,0.10,0.15,0.24,0.17,0.42,0.42
3,Crema Bio-Hydratante,0.41,0.29,0.13,1.00,0.29,0.35,0.33,0.27,0.20,0.53,0.30,0.32,0.28,0.21,0.31
4,Crema Clarant B3 Normal a Grasa,0.22,0.19,0.19,0.29,1.00,0.16,0.38,0.15,0.16,0.26,0.36,0.39,0.34,0.18,0.17
5,Crema C con extracto de pepino,0.26,0.35,0.19,0.35,0.16,1.00,0.29,0.22,0.16,0.38,0.25,0.26,0.25,0.28,0.35
6,Crema de día Rejuveness,0.23,0.34,0.16,0.33,0.38,0.29,1.00,0.25,0.21,0.30,0.32,0.33,0.85,0.22,0.31
7,Jabón Facial Oil Control,0.34,0.31,0.28,0.27,0.15,0.22,0.25,1.00,0.68,0.23,0.21,0.18,0.25,0.26,0.29
8,Jabón Facial Sensitive,0.27,0.24,0.41,0.20,0.16,0.16,0.21,0.68,1.00,0.16,0.19,0.30,0.22,0.38,0.34
9,Crema H con filtro solar,0.23,0.30,0.10,0.53,0.26,0.38,0.30,0.23,0.16,1.00,0.23,0.34,0.31,0.19,0.25


In [81]:
import pandas as pd
import numpy as np

Quiz_output_limit = 3
Similarity_output_limit = 5

print('TAKE A QUIZ...')
Mapping = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "Item_attribute_map_Ponds")
Description = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "Item_description_Ponds")
Quiz = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "Quiz_attribute_str_Ponds")
Matrix = pd.read_excel("Similarity_matrix_output_Ponds.xlsx")

selection_list = Quiz.Type.tolist()

option_list = []
for i, val in enumerate(Quiz.Question):
    options = Quiz.iloc[i].values.tolist()[2:]
    options_clean = [x for x in options if str(x) != 'nan']
    print(i+1, val, options_clean)
    
    if selection_list[i] != 'Multi':
        option_selected = input("Enter your option here: ")
        while option_selected.strip() not in options_clean:
            print("Please type from the list of options above.")
            option_selected = input("Type your option here: ")

        print("")
        if option_selected.strip() != "Skip" and option_selected.strip() != "Quit":
            option_list.append(option_selected.strip())
        if option_selected.strip() == "Quit":
            break
    else:
        option_selected_list = input("Enter your multiple options here (',' seperated): ")
        option_selected = option_selected_list.split(",")
        option_selected = [i.strip() for i in option_selected]
        while (set(option_selected).issubset(set(options_clean))) is False:
            print(f"Please type options from the above list with ',' seperated.")
            option_selected_list = input("Enter your multiple options here (',' seperated): ")
            option_selected = option_selected_list.split(",")
            option_selected = [i.strip() for i in option_selected]
            
        print("")
        for opt in option_selected:
            opt = opt.strip()
            if opt != "Skip" and opt != "Quit":
                option_list.append(opt)
            if opt == "Quit":
                break
        if opt == "Quit":
            break        

print("-"*60)
print("Selected attributes: ", option_list)
print("-"*60)
Mapping_score = Mapping[option_list]
Mapping['Score'] = Mapping_score.sum(axis=1) 
Mapping = Mapping.sort_values(by = 'Score',ascending = False).reset_index().drop('index', axis = 1)
print("\nRecommendation based on Quiz:")
print(Mapping[['Name', 'Score']].head(Quiz_output_limit))
Mapping[['Name', 'Score']].head(Quiz_output_limit).to_csv("products_quiz_output.csv")
print("-"*60)
for i in Mapping['Name'].head(Quiz_output_limit).tolist():
    print('\nRecommendation based on Similarity:')
    print(Matrix[['Name', i]].sort_values(by = i, ascending = False).reset_index(inplace = False).drop('index', axis = 1).drop([0]).head(Similarity_output_limit))

TAKE A QUIZ...
1 Skin Goal? ['Acne-free', 'Fairness', 'Younger', 'Spot Free', 'Skip', 'Quit']
Enter your multiple options here (',' seperated): Younger

2 Requirement? ['Oil-free', 'Hydrate', 'Fresh', 'Skip', 'Quit']
Enter your option here: Fresh

3 Outdoor or Indoor? ['UV-Protect', 'Rejuvenate', 'Skip', 'Quit']
Enter your multiple options here (',' seperated): Quit

------------------------------------------------------------
Selected attributes:  ['Younger', 'Fresh']
------------------------------------------------------------

Recommendation based on Quiz:
                      Name  Score
0            Loción C Dual    2.0
1           Agua Micelar C    2.0
2  Crema de día Rejuveness    2.0
------------------------------------------------------------

Recommendation based on Similarity:
                             Name  Loción C Dual
1           Loción Bio-Hydratante           0.40
2                     Toallitas C           0.40
3  Crema C con extracto de pepino           0.35
4   

In [70]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

Quiz_output_limit = 3
Similarity_output_limit = 10

print('TAKE A QUIZ...')
Mapping = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "ATH_mapping")
Description = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "ATH_desc")
Quiz = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "ATH_quiz")

selection_list = Quiz.Slection.tolist()

option_list = []
for i, val in enumerate(Quiz.Question):
    options = Quiz.iloc[i].values.tolist()[2:]
    options_clean = [x for x in options if str(x) != 'nan']
    print(i+1, val, options_clean)
    
    if selection_list[i] != 'Multi':
        option_selected = input("Enter your option here: ")
        while option_selected.strip() not in options_clean:
            print("Please type from the list of options above.")
            option_selected = input("Type your option here: ")

        print("")
        if option_selected.strip() != "Skip" and option_selected.strip() != "Quit":
            option_list.append(option_selected.strip())
        if option_selected.strip() == "Quit":
            break
    else:
        option_selected_list = input("Enter your multiple options here (',' seperated): ")
        option_selected = option_selected_list.split(",")
        option_selected = [i.strip() for i in option_selected]
        while (set(option_selected).issubset(set(options_clean))) is False:
            print(f"Please type options from the above list with ',' seperated.")
            option_selected_list = input("Enter your multiple options here (',' seperated): ")
            option_selected = option_selected_list.split(",")
            option_selected = [i.strip() for i in option_selected]
            
        print("")
        for opt in option_selected:
            opt = opt.strip()
            if opt != "Skip" and opt != "Quit":
                option_list.append(opt)
            if opt == "Quit":
                break
        if opt == "Quit":
            break        
print("Selected attributes: ", option_list)

Mapping_score = Mapping[option_list]
Mapping['Score'] = Mapping_score.sum(axis=1) 
Mapping = Mapping.sort_values(by = 'Score',ascending = False).reset_index().drop('index', axis = 1)
print("\nBased on Quiz:")
print(Mapping[['Name', 'Score']].head(Quiz_output_limit))
Mapping[['Name', 'Score']].head(Quiz_output_limit).to_csv("products_quiz_output.csv")

Quiz_output = Mapping['Name'].head(Quiz_output_limit).tolist()
Description['merged'] = Description['Description'].astype(str) + " " + Description['Ingredients'].astype(str) + " " + Description['Work'].astype(str) #+ " " + Description['Tutorial'].astype(str) + Description['Title_1'].astype(str) + ", " + Description['Title_2'].astype(str) + ", " + Description['Title_3'].astype(str) + ", " + Description['Title_4'].astype(str)  + ", " + Description['Title_5'].astype(str) 

Description_name = Description['Name'].tolist()
Description_desc = Description['merged'].tolist()
Description_clean = []
for i in Description_desc:
    Description_clean.append(i.replace('\n', ' ').replace('\r', ' ').replace('\\', ' ').replace('nan', ' ').replace('/', ' '))

Quiz_output_desc = []
for i in Quiz_output:
    if i in Description_name:
        Quiz_output_desc.append(Description_clean[Description_name.index(i)])

similar_products = pd.DataFrame()
for ii,X in enumerate(Quiz_output_desc):
    similarity_index = []
    for j,Y in enumerate(Description_clean):
        #Removing symbols
        symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
        for i in symbols:
            Y = Y.replace(i, ' ')
            X = X.replace(i, ' ')

        X_list = word_tokenize(X.lower())  
        Y_list = word_tokenize(Y.lower()) 

        # sw contains the list of stopwords 
        sw = stopwords.words('english')  
        l1 =[];l2 =[] 

        # remove stop words from the string 
        X_set = {w for w in X_list if not w in sw}  
        Y_set = {w for w in Y_list if not w in sw} 

        #Stemming and remove words with length 1 and 2 letters
        X_set_new, Y_set_new = {''}, {''}
        for w in X_set:
            if len(w) > 2:
                X_set_new.add(stemmer.stem(w))
        for w in Y_set:
            if len(w) > 2:
                Y_set_new.add(stemmer.stem(w))    

        # form a set containing keywords of both strings  
        rvector = X_set_new.union(Y_set_new)  
        for w in rvector: 
            if w in X_set_new: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set_new: l2.append(1) 
            else: l2.append(0) 
        c = 0

        # cosine formula  
        for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        similarity_index.append(round(cosine, 2))
        
    simil_pred = pd.DataFrame({'Name': Description_name, 'SI' : similarity_index})
    simil_pred = simil_pred.sort_values(by = 'SI',ascending = False)
    simil_pred = simil_pred.reset_index(inplace = False).drop('index', axis = 1) 
    simil_pred = simil_pred.drop([0])
    similar_products = similar_products.append(simil_pred.iloc[:10])
#     print(f'\nTop {Similarity_output_limit} Similar products of "{Quiz_output[ii]}"')
#     print(simil_pred.head(Similarity_output_limit))
    
similar_products = similar_products.sort_values(by = 'SI',ascending = False)
similar_products = similar_products.drop_duplicates(['Name'], keep='first')
similar_products =  similar_products.reset_index().drop('index', axis = 1)

print("\nBased on Similarity:")
print(similar_products.head(Similarity_output_limit))
similar_products.head(Similarity_output_limit).to_csv("products_similarity_output.csv")

TAKE A QUIZ...
1 Length? ['Short', 'Medium', 'Long', 'Skip', 'Quit']
Enter your multiple options here (',' seperated): Quit

Selected attributes:  []

Based on Quiz:
                  Name  Score
0            pixie cut    0.0
1       Fishtail braid    0.0
2  Double dutch braids    0.0

Based on Similarity:
                  Name    SI
0  Double dutch braids  0.53
1       Fishtail braid  0.53
2             Big afro  0.44
3       Bob box braids  0.42
4          Bantu knots  0.40
5      Slick back hair  0.38
6             Wavy bob  0.35
7          Bob haircut  0.35
8        Victory rolls  0.34
9   Natural hair twist  0.33


In [67]:
Description['Products_merged'] = Description['Title_1'].astype(str) + ", " + Description['Title_2'].astype(str) + ", " + Description['Title_3'].astype(str) + ", " + Description['Title_4'].astype(str)  + ", " + Description['Title_5'].astype(str)  
Description_products = Description['Products_merged'].tolist()
Description_tutorials = Description['Tutorial'].tolist()

Quiz_output_products = []
Quiz_output_tutorials = []
for i in Mapping['Name'].head(Quiz_output_limit).tolist():
    if i in Description_name:
        Quiz_output_tutorials.append(Description_tutorials[Description_name.index(i)])
        Quiz_output_products.append(Description_products[Description_name.index(i)])

grouping = pd.DataFrame({"Name": Mapping['Name'].head(Quiz_output_limit).tolist(), "Tutorials": Quiz_output_tutorials,
                        "Products": Quiz_output_products})
grouping

,Name,Tutorials,Products
0,Balayage,How to Get a Celeb-Worthy Balayage Hair Color,"TRESemmé Keratin Smooth Colour Shampoo, nan, n..."
1,Sleek bob,Flat iron hair: how to get super sleek strands,"TRESemmé REPAIR & PROTECT 7 PRE-STYLING SPRAY,..."
2,Crimped,Tutorial: Half-Crimped Hair,"NEXXUS CITY SHIELD SHAMPOO, NEXXUS CITY SHIELD..."


In [71]:
import pandas as pd

Description = pd.read_excel("Quiz_questions_template.xlsx", sheet_name = "Item_description_Ponds")
Description['merged'] = Description['Description'].astype(str) + " " + Description['Ingredients'].astype(str) + " " + Description['Work'].astype(str) #+ " " + Description['Tutorial'].astype(str) + Description['Title_1'].astype(str) + ", " + Description['Title_2'].astype(str) + ", " + Description['Title_3'].astype(str) + ", " + Description['Title_4'].astype(str)  + ", " + Description['Title_5'].astype(str) 

Description_name = Description['Name'].tolist()
Description_desc = Description['merged'].tolist()
Description_clean = []
for i in Description_desc:
    Description_clean.append(i.replace('\n', ' ').replace('\r', ' ').replace('\\', ' ').replace('nan', ' ').replace('/', ' '))

similar_products = pd.DataFrame({'Name': Description_name})
for ii,X in enumerate(Description_clean):
    similarity_index = []
    for j,Y in enumerate(Description_clean):
        #Removing symbols
        symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
        for i in symbols:
            Y = Y.replace(i, ' ')
            X = X.replace(i, ' ')

        X_list = word_tokenize(X.lower())  
        Y_list = word_tokenize(Y.lower()) 

        # sw contains the list of stopwords 
        sw = stopwords.words('english')  
        l1 =[];l2 =[] 

        # remove stop words from the string 
        X_set = {w for w in X_list if not w in sw}  
        Y_set = {w for w in Y_list if not w in sw} 

        #Stemming and remove words with length 1 and 2 letters
        X_set_new, Y_set_new = {''}, {''}
        for w in X_set:
            if len(w) > 2:
                X_set_new.add(stemmer.stem(w))
        for w in Y_set:
            if len(w) > 2:
                Y_set_new.add(stemmer.stem(w))    

        # form a set containing keywords of both strings  
        rvector = X_set_new.union(Y_set_new)  
        for w in rvector: 
            if w in X_set_new: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set_new: l2.append(1) 
            else: l2.append(0) 
        c = 0

        # cosine formula  
        for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        similarity_index.append(round(cosine, 2))

    similar_products[f'{Description_name[ii]}'] = similarity_index
similar_products.to_csv("Similarity_matrix_output_Ponds.csv", index=False)
similar_products

,Name,Loción Bio-Hydratante,Loción C Dual,Agua Micelar C,Crema Bio-Hydratante,Crema Clarant B3 Normal a Grasa,Crema C con extracto de pepino,Crema de día Rejuveness,Jabón Facial Oil Control,Jabón Facial Sensitive,Crema H con filtro solar,Crema S Nutritiva,Crema Sensitive con extracto de manzanilla,Crema de noche Rejuveness,Toallitas Sensitive,Toallitas C
0,Loción Bio-Hydratante,1.00,0.40,0.29,0.41,0.22,0.26,0.23,0.34,0.27,0.23,0.19,0.20,0.24,0.26,0.29
1,Loción C Dual,0.40,1.00,0.29,0.29,0.19,0.35,0.34,0.31,0.24,0.30,0.17,0.22,0.28,0.31,0.40
2,Agua Micelar C,0.29,0.29,1.00,0.13,0.19,0.19,0.16,0.28,0.41,0.10,0.15,0.24,0.17,0.42,0.42
3,Crema Bio-Hydratante,0.41,0.29,0.13,1.00,0.29,0.35,0.33,0.27,0.20,0.53,0.30,0.32,0.28,0.21,0.31
4,Crema Clarant B3 Normal a Grasa,0.22,0.19,0.19,0.29,1.00,0.16,0.38,0.15,0.16,0.26,0.36,0.39,0.34,0.18,0.17
5,Crema C con extracto de pepino,0.26,0.35,0.19,0.35,0.16,1.00,0.29,0.22,0.16,0.38,0.25,0.26,0.25,0.28,0.35
6,Crema de día Rejuveness,0.23,0.34,0.16,0.33,0.38,0.29,1.00,0.25,0.21,0.30,0.32,0.33,0.85,0.22,0.31
7,Jabón Facial Oil Control,0.34,0.31,0.28,0.27,0.15,0.22,0.25,1.00,0.68,0.23,0.21,0.18,0.25,0.26,0.29
8,Jabón Facial Sensitive,0.27,0.24,0.41,0.20,0.16,0.16,0.21,0.68,1.00,0.16,0.19,0.30,0.22,0.38,0.34
9,Crema H con filtro solar,0.23,0.30,0.10,0.53,0.26,0.38,0.30,0.23,0.16,1.00,0.23,0.34,0.31,0.19,0.25


In [69]:
for i in Mapping['Name'].head(Quiz_output_limit).tolist():
    print('\nSimilarity recommendation:')
    print(similar_products[['Name', i]].sort_values(by = i, ascending = False).reset_index(inplace = False).drop('index', axis = 1).drop([0]).head(5))


Similar recommendation:
                  Name  Balayage
1       Headband braid      0.28
2             Dip dyed      0.28
3  Double dutch braids      0.26
4   Natural hair twist      0.24
5       Bob box braids      0.23

Similar recommendation:
                     Name  Sleek bob
1                 Lob bob       0.46
2  Natural hair perm rods       0.39
3             Bob haircut       0.34
4       Pineapple frohawk       0.33
5           Victory rolls       0.33

Similar recommendation:
                Name  Crimped
1           Rose bun     0.52
2              Braid     0.43
3          Donut bun     0.39
4  Faux french braid     0.39
5    Waterfall braid     0.39


In [86]:
df = pd.DataFrame(pd.read_json('ponds.json', orient='brand'))
df

,locales,status,message
0,"{'localeName': 'es-US', 'brand': {'name': 'pon...",success,Feeds generated successfully
1,"{'localeName': 'en-US', 'brand': {'name': 'pon...",success,Feeds generated successfully
